# Encontrando la Estrategia Optima

Ya se ha visto que la estrategia Mean-Reversion utilizando Bandas de Bollinger requiere de 2 parámetros: la ventana de la SMA y el número de desviaciones estándar para calcular las bandas superior e inferior.

Similarmente como se hizo con la estrategia SMA, es posible optimizar los parámetros de la estrategia Mean-Reversion para maximizar los retornos.

In [1]:
# importar las librerías
import pandas as pd
import numpy as np

In [2]:
# importar los datos
df = pd.read_csv("intraday.csv", parse_dates = ["time"], index_col = "time")

In [3]:
df

,price
time,
2018-01-01 22:00:00+00:00,1.201205
2018-01-02 04:00:00+00:00,1.207055
2018-01-02 10:00:00+00:00,1.204440
2018-01-02 16:00:00+00:00,1.205800
2018-01-02 22:00:00+00:00,1.204690
...,...
2019-12-29 22:00:00+00:00,1.119920
2019-12-30 04:00:00+00:00,1.119940
2019-12-30 10:00:00+00:00,1.120095


In [6]:
''' Ahora se define la función que permite optimizar la estrategia. La función requiere 2 parámetros que se le pasan en una lista'''
def run_strategy(boll):
    data = df.copy()
    data["returns"] = np.log(data.price.div(data.price.shift(1)))

    data["SMA"] = data["price"].rolling(boll[0]).mean()
    data["Lower"] = data["SMA"] - data["price"].rolling(boll[0]).std() * boll[1]
    data["Upper"] = data["SMA"] + data["price"].rolling(boll[0]).std() * boll[1]
    data.dropna(inplace=True)

    data["distance"] = data.price - data.SMA

    data["position"] = np.where(data.price < data.Lower, 1, np.nan)
    data["position"] = np.where(data.price > data.Upper, -1, data["position"])
    data["position"] = np.where(data.distance * data.distance.shift(1) < 0, 0, data["position"])
    data["position"] = data.position.ffill().fillna(0)

    data["strategy"] = data.position.shift(1) * data["returns"]
    data.dropna(inplace = True)
    
    
    return -data[["returns", "strategy"]].sum().apply(np.exp)[1]


In [7]:
# se corre la estrategia con algún parámetro inicial
run_strategy((30, 2))

C:\Users\marioL\AppData\Local\Temp\ipykernel_15848\1890563592.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return -data[["returns", "strategy"]].sum().apply(np.exp)[1]


np.float64(-1.0781362437114321)

In [8]:
# importar la librería para optimizar
from scipy.optimize import brute

In [9]:
# llamada a la optimización se le pasa la función y el rango de los parámetros a optimizar
brute(run_strategy, ((20, 100, 1), (1, 5, 1)), finish = None)

C:\Users\marioL\AppData\Local\Temp\ipykernel_15848\1890563592.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return -data[["returns", "strategy"]].sum().apply(np.exp)[1]
C:\Users\marioL\AppData\Local\Temp\ipykernel_15848\1890563592.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return -data[["returns", "strategy"]].sum().apply(np.exp)[1]
C:\Users\marioL\AppData\Local\Temp\ipykernel_15848\1890563592.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by p

array([58.,  1.])

El resultado anterior muestra que los valores óptimos son un período de ventana de 58 y las bandas de bollinger calculadas a 1 desviación estándar. Estos valores se deben usar para probar el backtesting que está en 3.3.2_5-mean_reversion_strategy.ipynb